In [2]:
import pandas as pd
import glob
import os
from pprint import pprint

In [6]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\ID\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [7]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\ID\20161108__id__general__precinct.csv


In [8]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\ID\20160308__id__primary__presidential__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\ID\20160517__id__primary__precinct.csv


In [10]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
            
        df = df.drop_duplicates()

        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
pri_combined_df["precinct"] = pri_combined_df["county"].astype(str) + pri_combined_df["precinct"].astype(str)
pri_combined_df["precinct"] = pri_combined_df["precinct"].str.upper()
pri_combined_df


,county,precinct,office,district,party,candidate,votes
0,ADA,ADA1401,President,NaN,REP,Jeb Bush,0.0
1,ADA,ADA1402,President,NaN,REP,Jeb Bush,0.0
2,ADA,ADA1403,President,NaN,REP,Jeb Bush,1.0
3,ADA,ADA1404,President,NaN,REP,Jeb Bush,1.0
4,ADA,ADA1405,President,NaN,REP,Jeb Bush,1.0
...,...,...,...,...,...,...,...
12488,WASHINGTON,WASHINGTON07 MIDVALE,President,NaN,REP,Donald J. Trump,96.0
12489,WASHINGTON,WASHINGTON08 CAMBRIDGE,President,NaN,REP,Donald J. Trump,72.0
12490,WASHINGTON,WASHINGTON09 PIONEER,President,NaN,REP,Donald J. Trump,52.0
12491,WASHINGTON,WASHINGTON10 SUNNYSIDE,President,NaN,REP,Donald J. Trump,88.0


In [12]:
pri_combined_df["party"].value_counts(dropna=False)

party
REP    12493
Name: count, dtype: int64

In [13]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,ADA1401,REP,Jeb Bush,0.0
1,ADA1402,REP,Jeb Bush,0.0
2,ADA1403,REP,Jeb Bush,1.0
3,ADA1404,REP,Jeb Bush,1.0
4,ADA1405,REP,Jeb Bush,1.0
...,...,...,...,...
12488,WASHINGTON07 MIDVALE,REP,Donald J. Trump,96.0
12489,WASHINGTON08 CAMBRIDGE,REP,Donald J. Trump,72.0
12490,WASHINGTON09 PIONEER,REP,Donald J. Trump,52.0
12491,WASHINGTON10 SUNNYSIDE,REP,Donald J. Trump,88.0


In [14]:
# primary_data.loc[:,"party"] = (
#     primary_data["party"]
#     .replace({
#         "Democratic": "DEM",
#         "Republican": "REP",
#         "Green-rainbow": "GRN"
#     })
#     .fillna("IND")
# )

primary_data["party"].value_counts(dropna=False)

party
REP    12493
Name: count, dtype: int64

In [15]:
primary_data["candidate"].value_counts(dropna=False)

candidate
Jeb Bush           961
Ben Carson         961
Chris Christie     961
Ted Cruz           961
Carly Fiorina      961
Lindsey Graham     961
Mike Huckabee      961
John R. Kasich     961
Peter Messina      961
Rand Paul          961
Marco Rubio        961
Rick Santorum      961
Donald J. Trump    961
Name: count, dtype: int64

In [16]:
# Cleaning Candidates
primary_data = primary_data[~primary_data["candidate"].isin(["Blank Votes", "All Others","Total Votes Cast","No Preference"])] 

primary_data["candidate"].value_counts(dropna=False)

candidate
Jeb Bush           961
Ben Carson         961
Chris Christie     961
Ted Cruz           961
Carly Fiorina      961
Lindsey Graham     961
Mike Huckabee      961
John R. Kasich     961
Peter Messina      961
Rand Paul          961
Marco Rubio        961
Rick Santorum      961
Donald J. Trump    961
Name: count, dtype: int64

In [17]:
primary_data["candidate"].unique()
candidate_party_map = (
    primary_data.dropna(subset=["candidate", "party"])
                .set_index("candidate")["party"]
                .to_dict()
)
print(candidate_party_map)

{'Jeb Bush': 'REP', 'Ben Carson': 'REP', 'Chris Christie': 'REP', 'Ted Cruz': 'REP', 'Carly Fiorina': 'REP', 'Lindsey Graham': 'REP', 'Mike Huckabee': 'REP', 'John R. Kasich': 'REP', 'Peter Messina': 'REP', 'Rand Paul': 'REP', 'Marco Rubio': 'REP', 'Rick Santorum': 'REP', 'Donald J. Trump': 'REP'}


In [18]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

candidate_column,precinct,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CHRISTIE,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GRAHAM,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_MESSINA,pri_rep_PAUL,pri_rep_RUBIO,pri_rep_SANTORUM,pri_rep_TRUMP
0,ADA1401,0.0,5.0,1.0,219.0,0.0,0.0,0.0,35.0,0.0,0.0,66.0,1.0,139.0
1,ADA1402,0.0,6.0,0.0,192.0,0.0,0.0,0.0,53.0,0.0,3.0,73.0,0.0,160.0
2,ADA1403,1.0,3.0,0.0,75.0,1.0,0.0,0.0,26.0,0.0,0.0,18.0,0.0,53.0
3,ADA1404,1.0,12.0,2.0,310.0,0.0,0.0,1.0,44.0,0.0,4.0,78.0,0.0,239.0
4,ADA1405,1.0,5.0,0.0,211.0,1.0,0.0,1.0,32.0,0.0,3.0,71.0,0.0,224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,WASHINGTON07 MIDVALE,1.0,9.0,0.0,120.0,0.0,0.0,0.0,11.0,0.0,1.0,14.0,0.0,96.0
957,WASHINGTON08 CAMBRIDGE,1.0,9.0,2.0,121.0,0.0,0.0,0.0,9.0,0.0,1.0,42.0,0.0,72.0
958,WASHINGTON09 PIONEER,2.0,7.0,1.0,46.0,1.0,0.0,0.0,16.0,0.0,0.0,24.0,2.0,52.0
959,WASHINGTON10 SUNNYSIDE,1.0,8.0,1.0,98.0,2.0,0.0,1.0,16.0,1.0,2.0,20.0,0.0,88.0


In [20]:
# Process general files
gen_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)

        # Select only president
        if 'office' in df.columns:
            df = df[df["office"] == "President" ]

        gen_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(gen_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
gen_combined_df["precinct"] = gen_combined_df["county"].astype(str) + gen_combined_df["precinct"].astype(str)
gen_combined_df["precinct"] = gen_combined_df["precinct"].str.upper()
gen_combined_df


,county,precinct,office,district,party,candidate,votes
0,ADA,ADA1401,President,NaN,IND,Darrell L. Castle,4
1,ADA,ADA1402,President,NaN,IND,Darrell L. Castle,3
2,ADA,ADA1403,President,NaN,IND,Darrell L. Castle,2
3,ADA,ADA1404,President,NaN,IND,Darrell L. Castle,13
4,ADA,ADA1405,President,NaN,IND,Darrell L. Castle,8
...,...,...,...,...,...,...,...
7683,WASHINGTON,WASHINGTON07 MIDVALE,President,NaN,REP,Donald J. Trump,403
7684,WASHINGTON,WASHINGTON08 CAMBRIDGE,President,NaN,REP,Donald J. Trump,398
7685,WASHINGTON,WASHINGTON09 PIONEER,President,NaN,REP,Donald J. Trump,288
7686,WASHINGTON,WASHINGTON10 SUNNYSIDE,President,NaN,REP,Donald J. Trump,359


In [21]:

general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]
general_data


,precinct,party,candidate,votes
0,ADA1401,IND,Darrell L. Castle,4
1,ADA1402,IND,Darrell L. Castle,3
2,ADA1403,IND,Darrell L. Castle,2
3,ADA1404,IND,Darrell L. Castle,13
4,ADA1405,IND,Darrell L. Castle,8
...,...,...,...,...
7683,WASHINGTON07 MIDVALE,REP,Donald J. Trump,403
7684,WASHINGTON08 CAMBRIDGE,REP,Donald J. Trump,398
7685,WASHINGTON09 PIONEER,REP,Donald J. Trump,288
7686,WASHINGTON10 SUNNYSIDE,REP,Donald J. Trump,359


In [22]:
general_data["candidate"].value_counts(dropna=False)

candidate
Darrell L. Castle         961
Hillary Rodham Clinton    961
Scott Copeland            961
Rocky De La Fuente        961
Gary Johnson              961
Evan McMullin             961
Jill Stein                961
Donald J. Trump           961
Name: count, dtype: int64

In [23]:
# general_data = general_data[~general_data["candidate"].isin(["All Others", "No Preference","Blank Votes","Total Votes Cast","Blanks","Blank","Write-In","BLANKS","Other Write-in","All Other Write-in Votes","Write-in votes","Write in","BLANK","Scattering Write-ins","SCATTERED","All Other Write Ins","Write-in"])] 
# general_data["candidate"] = (
#     general_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# general_data.loc[
#     (
#         general_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         general_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# # Fixing Sanders
# # general_data.loc[
# #     general_data["candidate"].str.contains("SANDERS", case=False, na=False),
# #     "candidate"
# # ] = "SANDERS"

# general_data.loc[
#     (
#         general_data["candidate"].str.contains("SANDERS", case=False, na=False) |
#         general_data["candidate"].str.contains("BERNIE S", case=False, na=False)
#     ),
#     "candidate"
# ] = "SANDERS"


general_data["candidate"].value_counts(dropna=False)

candidate
Darrell L. Castle         961
Hillary Rodham Clinton    961
Scott Copeland            961
Rocky De La Fuente        961
Gary Johnson              961
Evan McMullin             961
Jill Stein                961
Donald J. Trump           961
Name: count, dtype: int64

In [24]:
general_data["party"].value_counts(dropna=False)

party
IND    3844
DEM     961
CON     961
LIB     961
REP     961
Name: count, dtype: int64

In [25]:
# general_data["party"] = general_data.apply(
#     lambda row: candidate_party_map.get(row["candidate"], row["party"]) if pd.isna(row["party"]) else row["party"],
#     axis=1
# )
# def fill_party_from_general_data(row, df):
#     if pd.notna(row["party"]):
#         return row["party"]
    
#     # Try to find other rows with the same candidate and known party
#     matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
#     if not matches.empty:
#         return matches["party"].iloc[0]  # Return the first match's party
#     else:
#         return None  # Still unknown

# general_data["party"] = general_data.apply(
#     lambda row: fill_party_from_general_data(row, general_data),
#     axis=1
# )
# general_data["party"] = (
#     general_data["party"]
#     .replace({
#         "(Write-In)": "IND",
#         "Libertarian": "LIB",
#         "Democratic": "DEM",
#         "Republican":"REP",
#         "Green-rainbow": "GRN",
#         "Green-Rainbow": "GRN",
#         "Green and Rainbow": "GRN",
#     })
#     .fillna("IND")
# )

general_data["party"].value_counts(dropna=False)

party
IND    3844
DEM     961
CON     961
LIB     961
REP     961
Name: count, dtype: int64

In [26]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19148\1829951435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate_column"] = (


candidate_column,precinct,gen_con_COPELAND,gen_dem_CLINTON,gen_ind_CASTLE,gen_ind_FUENTE,gen_ind_MCMULLIN,gen_ind_STEIN,gen_lib_JOHNSON,gen_rep_TRUMP
0,ADA1401,0,206,4,0,63,10,35,857
1,ADA1402,2,292,3,1,86,8,49,796
2,ADA1403,2,121,2,1,41,5,18,289
3,ADA1404,2,317,13,7,148,13,79,1290
4,ADA1405,3,331,8,3,90,9,54,994
...,...,...,...,...,...,...,...,...,...
956,WASHINGTON07 MIDVALE,3,31,2,0,4,0,3,403
957,WASHINGTON08 CAMBRIDGE,0,68,2,0,29,2,8,398
958,WASHINGTON09 PIONEER,1,110,2,1,14,4,18,288
959,WASHINGTON10 SUNNYSIDE,4,48,2,1,24,1,15,359


In [28]:
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

combined

candidate_column,precinct,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CHRISTIE,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GRAHAM,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_MESSINA,...,gen_con_COPELAND,gen_dem_CLINTON,gen_ind_CASTLE,gen_ind_FUENTE,gen_ind_MCMULLIN,gen_ind_STEIN,gen_lib_JOHNSON,gen_rep_TRUMP,dem_primary_total,general_total
0,ADA1401,0,5,1,219,0,0,0,35,0,...,0,206,4,0,63,10,35,857,0,1175
1,ADA1402,0,6,0,192,0,0,0,53,0,...,2,292,3,1,86,8,49,796,0,1237
2,ADA1403,1,3,0,75,1,0,0,26,0,...,2,121,2,1,41,5,18,289,0,479
3,ADA1404,1,12,2,310,0,0,1,44,0,...,2,317,13,7,148,13,79,1290,0,1869
4,ADA1405,1,5,0,211,1,0,1,32,0,...,3,331,8,3,90,9,54,994,0,1492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,WASHINGTON07 MIDVALE,1,9,0,120,0,0,0,11,0,...,3,31,2,0,4,0,3,403,0,446
932,WASHINGTON08 CAMBRIDGE,1,9,2,121,0,0,0,9,0,...,0,68,2,0,29,2,8,398,0,507
933,WASHINGTON09 PIONEER,2,7,1,46,1,0,0,16,0,...,1,110,2,1,14,4,18,288,0,438
934,WASHINGTON10 SUNNYSIDE,1,8,1,98,2,0,1,16,1,...,4,48,2,1,24,1,15,359,0,454


In [29]:
combined.to_csv("ID.csv", index=False)
